In [ ]:
%reload_ext autoreload
%autoreload 2

Con las variables más importantes, entrenar un modelo Naive-Bayes y un modelo SVM (kernel=lineal)
- Usar 5 splits y 2 repeticiones
- Acometer el entrenamiento para diferentes hiperparámetros
- SVM: regularización (C, cost)
- Naive-Bayes (bernuilli): laplace smoothing